# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Perform an ETL process with multiple tables and create a single dataset

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this SQL query to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
```

In [4]:
import sqlite3
import pandas as pd 

con = sqlite3.connect('lego.db')
cur = con.cursor()
response = cur.execute("""SELECT name FROM sqlite_master
                                      WHERE type='table'
                                      ORDER BY name;""").fetchall()
response

[('product_details',),
 ('product_info',),
 ('product_pricing',),
 ('product_reviews',)]

In [8]:
for item in response:
    table = item[0]
    length = cur.execute("""SELECT count(*) from {};""".format(table)).fetchall()
    results = cur.execute("""SELECT * from {} limit 5;""".format(table)).fetchall()
    df = pd.DataFrame(results)
    df.columns = [x[0] for x in cur.description]
    print(table, length, '\n', df, '\n')

product_details [(744,)] 
    prod_id prod_desc prod_long_desc theme_name
0      630      blah       blahblah    Classic
1     2304      blah       blahblah     DUPLO®
2     7280      blah       blahblah       City
3     7281      blah       blahblah       City
4     7499      blah       blahblah       City 

product_info [(744,)] 
    prod_id  ages  piece_count                         set_name
0      630    4+            1                  Brick Separator
1     2304  1½-5            1     LEGO® DUPLO® Green Baseplate
2     7280  5-12            2      Straight & Crossroad Plates
3     7281  5-12            2  T-Junction & Curved Road Plates
4     7499  5-12           24     Flexible and Straight Tracks 

product_pricing [(10870,)] 
    prod_id country list_price
0    75823      US      29.99
1    75822      US      19.99
2    75821      US      12.99
3    21030      US      99.99
4    21035      US      79.99 

product_reviews [(744,)] 
    prod_id  num_reviews  play_star_rating revie

In [12]:
cur.execute("""SELECT * FROM product_details
                                JOIN product_info
                                USING(prod_id)
                                JOIN product_pricing
                                USING(prod_id)
                                JOIN product_reviews
                                USING(prod_id);""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
print(len(df))
df.head()

10870


,prod_id,prod_desc,prod_long_desc,theme_name,ages,piece_count,set_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,blah,blahblah,Angry Birds™,6-12,277,Bird Island Egg Heist,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,blah,blahblah,Angry Birds™,6-12,168,Piggy Plane Attack,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,blah,blahblah,Angry Birds™,6-12,74,Piggy Car Escape,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,blah,blahblah,Architecture,12+,1032,United States Capitol Building,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,blah,blahblah,Architecture,12+,744,Solomon R. Guggenheim Museum®,US,79.99,14.0,3.2,Challenging,4.6,4.1


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10870 entries, 0 to 10869
Data columns (total 14 columns):
prod_id              10870 non-null int64
prod_desc            10870 non-null object
prod_long_desc       10870 non-null object
theme_name           10870 non-null object
ages                 10870 non-null object
piece_count          10870 non-null int64
set_name             10870 non-null object
country              10870 non-null object
list_price           10870 non-null object
num_reviews          9449 non-null float64
play_star_rating     9321 non-null float64
review_difficulty    10870 non-null object
star_rating          9449 non-null float64
val_star_rating      9301 non-null float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.2+ MB


In [14]:
df.to_csv('Lego_data_merged.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity! In this lab, you successfully executed an ETL process to merge different tables!